# Paso A: Cortar segmentos del registro

In [ ]:
# -------------------------------------------------------------------------------------------------------- #

PATHFunciones = "/home/isabel/Dropbox/Proyectos/Papers/Romero-2022/";
HD = "/run/media/isabel/7bb4008d-08df-440b-9635-60c2187d8dfb/"
PATHBRWs = string( HD, "Data/Hipocampo/01-06-2022/BRWs");

push!( LOAD_PATH, PATHFunciones ); 

using Revisadas

FILESBRW = searchdir( PATHBRWs, "brw" );

i = 1;

FILEBRW = FILESBRW[ i ];
Variables = BRWDescription( FILEBRW );

PATHMain = pwd( );
PATHInfo = joinpath( PATHMain , "Info" );
PATHSegmentos = joinpath( PATHMain, "Segmentos" ); mkpath( PATHSegmentos );

@time cortar_cachos( Variables, PATHSegmentos );

println( "Archivo: ", FILEBRW, " terminado " );

# -------------------------------------------------------------------------------------------------------- #

# Paso B: Convertir a voltaje y remover saturaciones

In [ ]:
# -------------------------------------------------------------------------------------------------------- #

PATHFunciones = "/home/isabel/Dropbox/Proyectos/Papers/Romero-2022/";
HD = "/run/media/isabel/7bb4008d-08df-440b-9635-60c2187d8dfb/";
PATHMain = string( HD, "Data/Hipocampo/01-06-2022/Phase_10");

umbral_desviaciones = 2;
minimos_canales = 900;
maximos_canales = 2000;
ΔT = 500; # ms
umbral_arriba = 1000; # μV
umbral_abajo = -2500; # μV
porcentaje = 50; # %

push!( LOAD_PATH, PATHFunciones );

using Revisadas
using Plots
using JLD

PATHFotos = joinpath( PATHMain , "Fotos" );
PATHFiguras = joinpath( PATHMain, "Figuras" ); mkpath( PATHFiguras );
PATHFigurasNoSats = joinpath( PATHFiguras, "NoSats" ); mkpath( PATHFigurasNoSats );
PATHNoSat = joinpath( PATHMain, "NoSats" ); mkpath( PATHNoSat );
FILEVars = joinpath( PATHInfo, "variablesBRW.jld" );
FILEFoto = joinpath( PATHFotos, "Rebanada.png" );
FILESegmentos = searchdir( PATHSegmentos, "jld" );
FILESelections = joinpath( PATHInfo,  "Selections.jld" )

Variables = load( FILEVars )[ "Variables" ];
nChs = length( Variables[ "Layout" ] );
channels = 1:nChs;
day = splitpath( PATHMain )[ end - 1 ];

Selections = Array{ Int64 }( undef, nChs, length( FILESegmentos ) ); fill!( Selections, 0 );

@time for i = 1:length( FILESegmentos )

    FILESegmento = FILESegmentos[ i ];
    FILEfigure = joinpath( PATHFigurasNoSats, replace( basename( FILESegmento ), "jld" => "svg" ) );
    FILENoSat = replace( replace( FILESegmento, "Segmentos" => "NoSats" ), "BIN" => "NOSAT" );

    BIN = load( FILESegmento )[ "data" ];
    BIN = Digital2Analogue( Variables, BIN );

    BIN, _, _ = reparar_saturacion( umbral_arriba, umbral_abajo, porcentaje, BIN );
    BIN = reparar_rotacion( BIN );

    descartados = [ 1 ];

    STD, STD2 = ΔV( Variables, BIN, ΔT, descartados );

    Todos, t = Thresholding( Variables, STD );

    FIGUREFinal, metodo, Nmetodo = Figures4(
        Variables, Todos, t, minimos_canales, maximos_canales, STD, FILEFoto, FILESegmento, day, STD2 );

    Pesos = hcat( vcat( 0, STD ), vcat( reshape( t, 1, length( t ) ), Todos ) );
    index = Int.( Pesos[ 2:end, Nmetodo + 1 ] );

    seleccion = channels[ Bool.( index ) ];
    Selections[ seleccion, i ] .= 1;

    save( FILENoSat,
        "data", Float16.( BIN ),
        "metodo", metodo,
        "W", Pesos,
        "Seleccion", seleccion
    );

    savefig( FIGUREFinal, FILEfigure );

end

save( FILESelections, "Selections", Selections );

println( "All files ready" )

# -------------------------------------------------------------------------------------------------------- #

# Paso C: Seleccionar canales de interes y juntarlos en un solo archivo

In [ ]:
# -------------------------------------------------------------------------------------------------------- #

PATHFunciones = "/home/isabel/Dropbox/Proyectos/Papers/Romero-2022";
HD = "/run/media/isabel/7bb4008d-08df-440b-9635-60c2187d8dfb/";
PATHMain = string( HD, "Data/Hipocampo/01-06-2022/Phase_10");


push!( LOAD_PATH, PATHFunciones );

using Revisadas
using JLD
using Plots

PATHInfo = joinpath( PATHMain , "Info" );
PATHFiguras = joinpath( PATHMain, "Figuras" );
PATHNoSat = joinpath( PATHMain, "NoSats" ); 
PATHFotos = joinpath( PATHMain , "Fotos" );

cd( PATHInfo )

FILEVars = joinpath( PATHInfo, "variablesBRW.jld" );
FILEFiguraFinal = joinpath( PATHFiguras, "Seleccion_Canales.svg" );
FILEFiguraGrupos = joinpath( PATHFiguras, "FiguraGrupos.svg" );
FILEMatriz = joinpath( PATHInfo, "Matriz.jld" );
FILESNoSat = searchdir( PATHNoSat, "jld" );
FILESelections = joinpath( PATHInfo,  "Selections.jld" )
FILEFoto = joinpath( PATHFotos, "Rebanada.png" );

Variables = load( FILEVars )[ "Variables" ];
Selections = load( FILESelections )[ "Selections" ];
NRecFrames = Variables[ "NRecFrames" ][ 1 ];
nChs = length( Variables[ "Layout" ] );
channels = 1:nChs;

limite = 550; # μm 

MeaChs2ChIDsMatrix = reverse( reshape( channels, Variables[ "NCols" ], Variables[ "NRows" ] )', dims = 1 );

W = sum( Selections, dims = 2 );

Todos, t = Thresholding( Variables, Float64.( vec( W ) ) );

Figuras_FFinal9( MeaChs2ChIDsMatrix, Todos, FILEFiguraFinal ); 

Canales, FIGGrupos = FiguraGrupos( FILEFiguraGrupos, Todos, FILEVars );
# Canales, _ = FiguraGrupos( FILEFiguraGrupos, Todos, FILEVars ); # <--

Canales = sort( Int.( Canales ) );

Z = Z0( Canales ); F1 = Zplot( Z, :greys );

grupos, canales_solos = obtener_grupos( Canales );

# ----------------------------------

# Si hay mancha hay que quitarla o identificarla...

mancha = sort( length.( grupos ) )[ end - 1 ];

Todos = vcat( ( grupos[ length.( grupos ) .!= mancha ] )...);

Z = Z0( Todos ); F2 = Zplot( Z, :greys );

nChsCA = maximum( length.( grupos ) );

CA = grupos[ length.( grupos ) .== nChsCA ][ 1 ];

Z = Z0( CA ); F3 = Zplot( Z, :greys );

# buscar el miembro mas cercano del grupo CA

sueltos = setdiff( Todos, CA );

CACoo = Index2Coordinates( CA ); sueltosCoo = Index2Coordinates( sueltos );

D_Sueltos = Array{ Float64 }( undef, length( CA )*length( sueltos ), 2 ); fill!( D_Sueltos, 0 );

for i = 1:length( sueltos )
    
    for j = 1:length( CA )

        dist =
            sqrt( ( sueltosCoo[ i, 2 ] - CACoo[ j, 2 ] )^2
                + ( sueltosCoo[ i, 3 ] - CACoo[ j, 3 ] )^2 );
        
        D_Sueltos[ ( ( i - 1 )*length( CA ) + j ), : ] = [ sueltosCoo[ i, 1 ], ( dist * 42 ) ];
        
    end
end

SueltosSeleccionados = D_Sueltos[ :, 1 ][ findall( D_Sueltos[ :, 2 ] .<= limite ) ];

SueltosSeleccionados = Int.( unique( SueltosSeleccionados ) );
 
Z = Z0( union( CA, SueltosSeleccionados ) ); F4 = Zplot( Z, :greys );

# Rellenar CA

oldCA = copy( CA ); huecos = [ ]; Nhuecos = 6;

while !isempty( huecos ) || length( CA ) == length( oldCA )
    
    noCA = setdiff( collect( channels ), CA );

    noCAnoSueltos = setdiff( noCA, SueltosSeleccionados );

    nVecinos = zeros( Int, length( noCAnoSueltos ), 2 );

    for i = 1:length( noCAnoSueltos )
        
        _, v = neighborgs( noCAnoSueltos[ i ], 1 );

        nVecinos[ i, : ] = [ noCAnoSueltos[ i ], length( intersect( CA, v ) ) ];
    end

    huecos = nVecinos[ :, 1 ][ findall( nVecinos[ :, 2 ] .>= Nhuecos ) ];

    CA = vcat( CA, huecos );
    
end

Z = Z0( CA ); F5 = Zplot( Z, :greys );

# Rellenar Sueltos

oldSueltos = copy( SueltosSeleccionados ); #SueltosSeleccionados = oldSueltos;

huecos = [ ]; Nhuecos = 5;

while !isempty( huecos ) || length( SueltosSeleccionados ) == length( oldSueltos )
    
    noSueltos = setdiff( collect( channels ), SueltosSeleccionados );

    noSueltosnoCA = setdiff( noSueltos, CA );

    nVecinos = zeros( Int, length( noSueltosnoCA ), 2 );

    for i = 1:length( noSueltosnoCA )

        _, v = neighborgs( noSueltosnoCA[ i ], 1 );

        nVecinos[ i, : ] = [ noSueltosnoCA[ i ], length( intersect( SueltosSeleccionados, v ) ) ];
    end

    huecos = nVecinos[ :, 1 ][ findall( nVecinos[ :, 2 ] .>= Nhuecos ) ];

    SueltosSeleccionados = vcat( SueltosSeleccionados, huecos );
    
end

Z = Z0( SueltosSeleccionados ); F6 = Zplot( Z, :greys );

Z = Z0( union( CA, SueltosSeleccionados ) ); F7 = Zplot( Z, :greys );

F8 = plot( load( FILEFoto ),
        aspect_ratio = 1,
        axis = ( [ ], false ),
        wsize = ( 400, 400 ),
        titlefont = ( 10, "arial" ) );

F9 = FIGGrupos;

CASueltos = union( CA, SueltosSeleccionados );

grupos, canales_solos = obtener_grupos( CASueltos );

Z = zeros( Int, nChs );

for i = 1:length( grupos )
    Z[ grupos[ i ] ] .= i + 1;
end

Z[ canales_solos ] .= 1;

Z = reverse( reshape( Z, Variables[ "NCols" ], Variables[ "NRows" ] )', dims = 1 );

F11 = Zplot( Z, :twilight ); savefig( "Depuracion_penultima.svg")

# seleccionar los 2 grupos mas grandes sin CA y eliminar las islas que estan lejos

GDparcial = sort( length.( grupos ) )[ end-2:end-1 ]; # 2 grupos grandes que no son CA

GDparcial = vcat( union( grupos[ length.( grupos ) .== GDparcial[ 1 ] ], 
        grupos[ length.( grupos ) .== GDparcial[ 2 ] ] )... );

aux = grupos[length.(grupos) .!= maximum( length.(grupos) )];

aux = setdiff( vcat(aux...), GDparcial );

GDpotenciales = union( aux, canales_solos );

D_GDpotenciales = Array{ Float64 }( undef, length( GDparcial )*length( GDpotenciales ), 2 ); 

fill!( D_GDpotenciales, 0 );

GDparcialCoo = Index2Coordinates( Int.( GDparcial ) );

GDpotencialesCoo = Index2Coordinates( Int.( GDpotenciales ) );

for i = 1:length( GDpotenciales )
    
    for j = 1:length( GDparcial )

        dist =
            sqrt( ( GDpotencialesCoo[ i, 2 ] - GDparcialCoo[ j, 2 ] )^2
                + ( GDpotencialesCoo[ i, 3 ] - GDparcialCoo[ j, 3 ] )^2 );
        
        D_GDpotenciales[ ( ( i - 1 )*length( GDparcial ) + j ), : ] = [ 
            GDpotencialesCoo[ i, 1 ], ( dist * 42 ) ];
        
    end
end

limite = 900; #μm

GDpotencialesSeleccionados = D_GDpotenciales[ :, 1 ][ findall( D_GDpotenciales[ :, 2 ] .<= limite ) ];

GDpotencialesSeleccionados = Int.( unique( GDpotencialesSeleccionados ) );

GD = unique( union( GDpotencialesSeleccionados, GDparcial ) );

Z = zeros( Int, nChs ); Z[ GD ] = Z[ GD ] .+ 2; Z[ CA ] = Z[ CA ] .+ 1;

Z = reverse( reshape( Z, Variables[ "NCols" ], Variables[ "NRows" ] )', dims = 1 );

F10 = Zplot( Z, :ice ); savefig( "CAGDfinal.svg");

#plot( F1, F2, F3, F4, F5, F6, F7, F8, F9, layout = ( 3, 3 ), wsize = ( 900, 900 ) );
#plot( F8, F9, F1, F2, F4, F3, F5, F6, F7, layout = ( 3, 3 ), wsize = ( 900, 900 ) );

FF = plot( F8, F1, F9, F2, F4, F7, F5, F11, F10, layout = ( 3, 3 ), wsize = ( 900, 900 ) );

savefig( "Todas_partefinal.svg" );

Variables[ "CA" ] = CA; Variables[ "GD" ] = GD; save( FILEVars, "Variables", Variables );

#for GD

FILEMatriz = replace( FILEMatriz, "Matriz.jld" => "MatrizGD.jld" );
@time MatrizGD, mapa_indexGD = cortar_Matriz( GD, NRecFrames, FILESNoSat, FILEMatriz );

#for CA

FILEMatriz = replace( FILEMatriz, "GD" => "CA" );
@time MatrizCA, mapa_indexCA = cortar_Matriz( CA, NRecFrames, FILESNoSat, FILEMatriz );

# -------------------------------------------------------------------------------------------------------- #

# Paso D: Identificar espigas y guardarlas en un archivo .mat para su procesamiento posterior

In [ ]:
# -------------------------------------------------------------------------------------------------------- #

PATHSpikesCA = joinpath( PATHMain, "CAEspigas" ); mkpath( PATHSpikesCA );
PATHSpikesGD = joinpath( PATHMain, "GDEspigas" ); mkpath( PATHSpikesGD );

SamplingRate = Variables[ "SamplingRate" ][ ];


# Para hacer los filtrados y deteccion de espigas
window = 5; # ms tamaño de la ventana
displace = 1; # ms desplazamiento de la ventana
stokes = 0.5; # ms distancia mínima entre evento y evento detectado
stdmin = 4; # umbral de deteccion
w_pre = 1; # ms
w_post = 2; # ms

# Centrando las espigas
Icut = 0.5; # ms cuantos puntos
Lcut = 1; # ms

# preparando para filtrar
lF = 300; # Hz Frequencia limite izquierdo para el pasa bandas
HF = 3000; # Hz Frecuencia limite derecho para el pasa bandas
fac = 10; # Hz para la atenuacion entre las bandas
NYQ = floor( Int, SamplingRate/2 );
order = Int( floor( ( SamplingRate / lF ) / 5 ) );

# limite superior de deteccion
Vmax = 300; # μV

parametros = Dict(
        "window" => ms2frames( window, SamplingRate ),
        "displace" => ms2frames( displace, SamplingRate ),
        "stokes" => ms2frames( stokes, SamplingRate ),
        "w_pre" => ms2frames( w_pre, SamplingRate ),
        "w_post" => ms2frames( w_post, SamplingRate ),
        "Icut" => ms2frames( Icut, SamplingRate ),
        "Lcut" => ms2frames( Lcut, SamplingRate ),
        "stdmin" => stdmin,
        "lF" => lF,
        "HF" => HF,
        "fac" => fac,
        "Vmax" => Vmax,
        "NYQ" => NYQ,
        "order" => order
    );

# For CA

for A = 1:length( mapa_indexCA[ :, 1 ] );
    
    Ncanal = Int.( mapa_indexCA[ A, 2 ] );
    
    canal = MatrizCA[ A, : ];
    
    SpikesMat( Ncanal, canal, parametros, Variables, PATHSpikesCA )
        
end

# For GD

for A = 1:length( mapa_indexGD[ :, 1 ] );
    
    Ncanal = Int.( mapa_indexGD[ A, 2 ] );
    
    canal = MatrizGD[ A, : ];
    
    SpikesMat( Ncanal, canal, parametros, Variables, PATHSpikesGD )
        
end

# -------------------------------------------------------------------------------------------------------- #

# Paso E: Eliminar Redundancia

In [ ]:
# -------------------------------------------------------------------------------------------------------- #

PATHFunciones = "/home/isabel/Dropbox/Proyectos/Papers/Romero-2022";
HD = "/run/media/isabel/7bb4008d-08df-440b-9635-60c2187d8dfb/";
PATHMain = string( HD, "Data/Hipocampo/01-06-2022/Phase_10");

push!( LOAD_PATH, PATHFunciones );
using Revisadas

using JLD
using MAT
using StatsBase
using Plots
using Statistics
using LinearAlgebra

PATHInfo = joinpath( PATHMain , "Info" ); 
PATHTimes = joinpath( PATHMain, "Times" ); 
PATHFiguras = joinpath( PATHMain, "Figuras" ); 

FILEVars = joinpath( PATHInfo, "variablesBRW.jld" );
FILESTimes = searchdir( PATHTimes, "mat" );
FILEredundancia = joinpath( PATHInfo, "redundancia.jld" );
FILEAllInfoUnits = joinpath( PATHInfo, "AllInfoUnits_All.jld" );
FILEAllInfoUnits_no0 = joinpath( PATHInfo, "AllInfoUnits_no0.jld" );
FILEAllInfoUnits_noRed = joinpath( PATHInfo, "AllInfoUnits_noRed.jld" );
FILEBin = joinpath( PATHInfo, "Binarizacion.jld" );
FILEMeanSpikes = joinpath( PATHInfo, "MeanSpikes.mat" );
FIGUREMeanSpikes = joinpath( PATHFiguras, "MeanSpikes.svg" );

Variables = load( FILEVars )[ "Variables" ];
nChs = length( Variables[ "Layout" ] );

Canales = IntfromSting.( basename.( searchdir( PATHTimes, "mat" ) ) );
MeaChs2ChIDsMatrix = reverse( reshape( collect( 1:nChs ), 64, 64 )', dims = 1 );

factor = 42; # μm
d = 2; # ( Electrodos en la vecindad. ej. 1 == 8-vecindad, 2 == 24-vecindad )
pico = 19; # frame donde esta el valor minimo de voltaje, se puede sacar del "par" de cada espiga
factor_decaimiento = 2; # arbitrario...a ojo
rsoma = 12; # μm, mitad del diametro promedio de una celula del tejido

# -------------------------------------------------------------------------------------------------------- #
# Sacar los clusters y las espigas de los archivos times de matlab

j = 0; all_spikes = [ ]; all_cluster_class = [ ]; all_chs = [ ];

for k = 1:length( FILESTimes )

    FILETimes = FILESTimes[ k ]; temp = basename( FILETimes ); AUX = matread( FILETimes );

    n_cells = length( unique( AUX[ "cluster_class" ][ :, 1 ] ) );

    spikes = AUX[ "spikes" ];

    cluster_class = AUX[ "cluster_class" ];

    if n_cells > 1

        clus_num = unique( AUX[ "cluster_class" ][ :, 1 ] );

        for i = 1:n_cells

            j = j + 1;

            index_clus = findall( cluster_class[ :, 1 ] .== clus_num[ i ] );

            cc = cluster_class[ index_clus, : ];

            sp = spikes[ index_clus, : ];

            push!( all_spikes, sp ); push!( all_cluster_class, cc ); push!( all_chs, IntfromSting( temp ) );

        end
    else

        j = j + 1

        push!( all_spikes, spikes ); push!( all_cluster_class, cluster_class );

        push!( all_chs, IntfromSting( temp ) );

    end

end

save( FILEAllInfoUnits, "cluster_class", all_cluster_class, "spikes", all_spikes, "channels", all_chs );

println( "Sacar clusters excepto el 0, listo")

# quitar las espigas cluster 0 y sacar las cosas finales

all_times = [ ]; new_channels = [ ]; new_spikes = [ ];

for i = 1:length( all_cluster_class )

    temp = all_cluster_class[ i ];

    if Int( unique( temp[ :, 1 ] )[ ] ) != 0

        push!( all_times, temp[ :, 2 ] )

        push!( new_channels, all_chs[ i ] )

        push!( new_spikes, all_spikes[ i ] )

    end
end

save( FILEAllInfoUnits_no0, "times", all_times, "channels", new_channels, "spikes", new_spikes );

println( "Quitar cluster 0" )

# Eliminar la redundancia de la (d*2)-vecindad

EE = [ ];

for canal_central = 1:length( Canales )

    central = new_channels[ canal_central ];

    n_units = length( findall( new_channels .== central ) )

    for unit = 1:n_units # cuantas celulas hay en el canal central

        cc_central = all_times[ findall( new_channels .== central )[ unit ] ];

        sp_central = new_spikes[ findall( new_channels .== central )[ unit ] ];

        A, v = neighborgs( central, d ); # Vecinos del canal central

        vecinos = intersect( Canales, v ); # los que si pertenecen al clustering

        posiciones_vecinos = findall( new_channels .∈ [ vecinos ] );

        n_vecions = length( posiciones_vecinos );

        for n_vecion = 1:n_vecions

            vecino = new_channels[ posiciones_vecinos[ n_vecion ] ];

            cc_vecino = all_times[ posiciones_vecinos[ n_vecion ] ];

            sp_vecino = new_spikes[ posiciones_vecinos[ n_vecion ] ];

            maybes = intersect( cc_central, cc_vecino );

            posicion_sp_vecino = findall( cc_vecino .∈ [ maybes ] );

            posicion_sp_central = findall( cc_central .∈ [ maybes ] );

            voltage_central = sp_central[ posicion_sp_central, pico ];

            voltaje_vecino = sp_vecino[ posicion_sp_vecino, pico ];

            central_cartesian = [ ( d + 1 ), ( d + 1 ) ];

            vecino_cartesian = getindex.( findall( A .== vecino ), [ 1, 2 ] );

            dist =
                sqrt( ( central_cartesian[ 1 ] - vecino_cartesian[ 1 ] )^2
                    + ( central_cartesian[ 2 ] - vecino_cartesian[ 2 ] )^2 );

            distancia = dist*factor; # en micrometros

            for i = 1:length( maybes )

                x = [ voltage_central[ i ], voltaje_vecino[ i ] ];

                Φ1 = minimum( x );

                Φ2_1 = ( Φ1*rsoma )/( rsoma + distancia );

                Φ2_2 = Φ1 / ( ( distancia/20 )*factor_decaimiento );

                Φ2_real = maximum( x ); mas_menos = abs( Φ2_1 ) - abs( Φ2_2 );

                posible_valor = [
                    ( minimum( [ Φ2_1, Φ2_2 ] ) + mas_menos ), ( minimum( [ Φ2_1, Φ2_2 ] ) - mas_menos ) ];

                if maximum( posible_valor )> Φ2_real > minimum( posible_valor )

                    eliminar = findall( x .== Φ2_real )[ 1 ];
                    # 1 = voltaje central
                    # 2 = voltaje vecino

                    e1 = vcat(
                        posicion_sp_central[ i ], central, findall( new_channels .== central )[ unit ],
                        x[ 1 ] );

                    e2 = vcat( posicion_sp_vecino[ i ], vecino, posiciones_vecinos[ n_vecion ], x[ 2 ] );

                    if eliminar == 1

                        push!( EE, vcat( distancia, e1, e2 ) )

                    elseif eliminar == 2

                        push!( EE, vcat( distancia, e2, e1 ) )

                    end
                    # else si son distintas, no hagas nada
                end

            end

        end

    end

end

EE = Array( hcat( EE... )' ); save( FILEredundancia, "redundancia",  EE );

# se va, se queda

# espiga_no, canal_no, posicion_no, voltaje_no
# espiga_si, canal_si, posicion_si, voltaje_si

finales_eliminar = Int.( unique( EE[ :, 2:4 ], dims = 1 ) );

a = finales_eliminar[ :, 3 ]; definitivos = findall.( isequal.( unique( a ) ), [ a ] );

for j = 1:length( definitivos )

    p = finales_eliminar[ definitivos[ j ][ 1 ], 3 ];

    q = finales_eliminar[ definitivos[ j ], 1 ];

    new_spikes[ p ] = new_spikes[ p ][ 1:end .∉ [ q ], : ];

    all_times[ p ] = all_times[ p ][ 1:end .∉ [ q ], : ];

end

save( FILEAllInfoUnits_noRed, "cluster_class", all_times, "spikes", new_spikes, "channels", new_channels );

println( "Eliminar la n-vecindad: listo " );

# sacar espiga promedio para clustering

all_MS = zeros( length( new_spikes ), size( new_spikes[ 1 ], 2 ) ); thr = 0.5;

@time for i = 1:length( new_spikes )

    spikes = new_spikes[ i ];
    
    all_MS[ i, : ] = vec( mean( spikes, dims = 1 ) );
    #all_MS[ i, : ] = vec( meanspike( spikes, thr ) );
    
end

matwrite( FILEMeanSpikes, Dict( "data" => all_MS ) );

# -------------------------------------------------------------------------------------------------------- #

# Paso F: Coincidencias

In [ ]:
# Interneurons in the stratum lucidum of the rat hippocampus: an anatomical and electrophysiological 
# characterization. Spruston N, Lubke J, Frotscher M
# J Comp Neurol, 1997 Sep 1, 385 (3), pages: 427 - 440
# Max firing rate (Max F.R.): 40±20.8 Hz (3); 1 source

# Specialized electrophysiological properties of anatomically identified neurons in the hilar region of 
# the rat  fascia dentata. 
# Lubke J, Frotscher M, Spruston N
# J Neurophysiol, 1998 Mar, 79 (3), pages: 1518 - 1534
# PMID: 9497429;

# Max firing rate ( Max F.R. ): 72 ± 8 Hz (16); 1 source

# Between 100 and 400 μm, the average mossy fiber action potential conduction velocity 
# was 0.27 ± 0.02 m s−1. 
# Over this same range, CA3 axonal conduction velocity conduction velocity is 0.38 ± 0.02 m s−1 
 
# High Threshold, Proximal Initiation, and Slow Conduction Velocity of Action Potentials 
# in Dentate Granule Neuron Mossy Fibers Geraldine J. Kress, Margaret J. Dowling, Julian P. Meeks, 
# and Steven Mennerick 2008 DOI:10.1152/jn.90295.2008

# -------------------------------------------------------------------------------------------------------- #

PATHFunciones = "/home/isabel/Dropbox/Proyectos/Papers/Romero-2022";
HD = "/run/media/isabel/7bb4008d-08df-440b-9635-60c2187d8dfb/";
PATHMain = string( HD, "Data/Hipocampo/01-06-2022/Phase_10");

push!( LOAD_PATH, PATHFunciones );
using Revisadas

using JLD
using Plots


@inline distancia( x1, x2, y1, y2, factor ) = ( sqrt( ( x2 - x1 )^2 + ( y2 - y1 )^2 ) )*factor;
@inline t( v, d ) = ( ( d / 1000000 ) / v ); # v(m/s), d(μm)

PATHInfo = joinpath( PATHMain , "Info" ); 
PATHFigurasGenerales = joinpath( PATHMain, "Figuras/Generales" ); mkpath( PATHFigurasGenerales );

cd( PATHFigurasGenerales );

FILEAllInfoUnits_noRed = joinpath( PATHInfo, "AllInfoUnits_noRed.jld" );
FILEVars = joinpath( PATHInfo, "variablesBRW.jld" );

Variables = load( FILEVars )[ "Variables" ];
TEMP = load( FILEAllInfoUnits_noRed );

NRecFrames = Variables[ "NRecFrames" ][ 1 ];
SamplingRate = Variables[ "SamplingRate" ][ 1 ];
nChs = length( Variables[ "Layout" ] );

segundos = ceil( Int, NRecFrames/SamplingRate );
MeaChs2ChIDsMatrix = reverse( reshape( collect( 1:nChs ), 64, 64 )', dims = 1 );

factor = 42; # μm
dims_chip = 2.7; # mm

#restriccion = floor( sqrt( dims_chip^2 + dims_chip^2 )*1000, digits = 3 ); # μm

restriccion = 1000; # μm

times = TEMP[ "cluster_class" ]; spikes = TEMP[ "spikes" ]; channels = Int.( TEMP[ "channels" ] );

In [ ]:
# -------------------------------------------------------------------------------------------------------- #
# separar canales GD y CA

CA = Variables[ "CA" ];
GD = Variables[ "GD" ];

CAtimes = times[ channels .∈ [ CA ] ];
CAspikes = spikes[ channels .∈ [ CA ] ];
CAchannels = channels[ channels .∈ [ CA ] ];

GDtimes = times[ channels .∈ [ GD ] ];
GDspikes = spikes[ channels .∈ [ GD ] ];
GDchannels = channels[ channels .∈ [ GD ] ];

CAFrecuencias = size.( CAspikes, 1) / segundos;

F1 = histogram( CAFrecuencias,
    leg = :none,
    xlabel = "Frequency ( Hz )",
    ylabel = " n cells "
);
savefig( "FreqCA.svg" )

GDFrecuencias = size.( GDspikes, 1) / segundos;

F1 = histogram( GDFrecuencias,
    leg = :none,
    xlabel = "Frequency ( Hz )",
    ylabel = " n cells "
); savefig( "FreqGD.svg" );

# -------------------------------------------------------------------------------------------------------- #

In [41]:
# -------------------------------------------------------------------------------------------------------- #
# Considerando velocidad de transmision de celula granular ( G -> P )

vGm = 0.27 - 0.02; vGM = 0.27 + 0.02; # m/sec

ch_MG = [ ]; d_MG = [ ]; n_MG =  [ ]; all_MG = [ ];

@time for i = 1:length( GDchannels )
    
    cT = GDchannels[ i ]; x1y1 = CartesianIndices( MeaChs2ChIDsMatrix )[ cT ];
    
    for j = 1:length( CAchannels )
        
        cB = CAchannels[ j ]; x2y2 = CartesianIndices( MeaChs2ChIDsMatrix )[ cB ];

        d = floor( distancia( x1y1[ 1 ], x2y2[ 1 ], x1y1[ 2 ], x2y2[ 2 ], factor ), digits = 3 ); 

        rango = [ t( vGm, d ), t( vGM, 2*d ) ].*1000; # msec

        tT = floor.( GDtimes[ i ], digits = 3 ); tB = floor.( CAtimes[ j ], digits = 3 );

        MG = [ ];

        for k = 1:length( tT )

            A = tT[ k ] + minimum( rango ); B = tT[ k ] + maximum( rango );

            if !isempty( findall( B .>= tB .>= A ) )

                push!( MG,  tT[ k ], tB[ findall( B .>= tB .>= A ) ] );

            end

        end

        if !isempty( MG )

            push!( n_MG, length( MG ) ); push!( d_MG, d );

            push!( ch_MG, [ i, j, cT, cB ] ); push!( all_MG, MG );

        end
        
    end
    
end
# -------------------------------------------------------------------------------------------------------- #

G = Dict(
    
    "canales"       => ch_MG,
    "distancias"    => d_MG,
    "coincidencias" => n_MG,
    "tiempos"       => all_MG,
    "velocidad"     => [ vGm, vGM ]
    
    ); save( "granulares.jld", "data", G );

# -------------------------------------------------------------------------------------------------------- #

382.060892 seconds (2.56 G allocations: 627.038 GiB, 10.38% gc time)


In [45]:
# -------------------------------------------------------------------------------------------------------- #
# Considerando velocidad de transmision de celula piramidal

vPm = 0.38 - 0.02; vPM = 0.38 + 0.02;  # m/sec


ch_MP = [ ]; d_MP = [ ]; n_MP =  [ ]; all_MP = [ ];

@time for i = 1:( length( CAchannels ) - 1 )
    
    cT = CAchannels[ i ]; x1y1 = CartesianIndices( MeaChs2ChIDsMatrix )[ cT ];
    
    for j = 2:length( CAchannels )
        
        if i != j
            
            cB = CAchannels[ j ]; x2y2 = CartesianIndices( MeaChs2ChIDsMatrix )[ cB ];

            d = floor( distancia( x1y1[ 1 ], x2y2[ 1 ], x1y1[ 2 ], x2y2[ 2 ], factor ), digits = 3 ); 

            rango = [ t( vPm, d ), t( vPM, 2*d ) ].*1000; # msec

            tT = floor.( CAtimes[ i ], digits = 3 ); tB = floor.( CAtimes[ j ], digits = 3 );

            MP = [ ];

            for k = 1:length( tT )

                A = tT[ k ] + minimum( rango ); B = tT[ k ] + maximum( rango );

                if !isempty( findall( B .>= tB .>= A ) )

                    push!( MP,  tT[ k ], tB[ findall( B .>= tB .>= A ) ] );

                end

            end

            if !isempty( MP )

                push!( n_MP, length( MP ) ); push!( d_MP, d );

                push!( ch_MP, [ i, j, cT, cB ] ); push!( all_MP, MP );

            end

        end
        
    end
    
end
# -------------------------------------------------------------------------------------------------------- #

P = Dict(
    
    "canales"       => ch_MP,
    "distancias"    => d_MP,
    "coincidencias" => n_MP,
    "tiempos"       => all_MP,
    "velocidad"     => [ vPm, vPM ]
    
); save( "piramidales.jld", "data", P );

# -------------------------------------------------------------------------------------------------------- #

2659.456420 seconds (13.83 G allocations: 3.257 TiB, 30.57% gc time)
